In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
from image_utils import *

%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research
#%cd /content/gdrive/My\ Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/
import sys

!pip install tensorflow-graphics

path_to_utils = '/content/gdrive/My Drive/Accident_Anticipation/EXTRACT_PHYSICS_NOTEBOOKS/utils'
sys.path.append(path_to_utils)
from distance_map_processing_utils import bounding_box_conversion, retrieve_distance_dir, binary_rectangle, retrieve_distance_dir
from distance_map_processing_utils import retrieve_frame_dir1, obtain_keys, obtain_keys1, h, sort_dir, find_bb_conv, conv_bb_based_off_existing_crop

from __future__ import absolute_import, division, print_function

from absl import logging
import os
import cv2
import re
import time
import glob
from matplotlib import pyplot as plt

import numpy as np
from PIL import Image

import tensorflow as tf
from depth_from_video_in_the_wild import model
from scipy import stats

train_num = 126
test_num = 46

## import function


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research/depth_from_video_in_the_wild
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/google_distance_code/google-research


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def shift_indices(batch_det_new):
  for batch in range(10):
    cur_batch = batch_det_new[batch, :, :, -1]
    cur_batch = cur_batch[cur_batch != 0]
    try:
      lowest_val = np.min(cur_batch)
    except:
      lowest_val = 1
    cur_batch[cur_batch != 0] = cur_batch[cur_batch != 0] + 1 - lowest_val
    batch_det_new[batch, :, :, -1][batch_det_new[batch, :, :, -1] != 0] = cur_batch
    unique_vals = np.unique(batch_det_new[batch, :, :, -1])

    # print(unique_vals)
    consecutive_vals = np.arange(len(unique_vals))

    for k in range(len(unique_vals)):
      if unique_vals[k] == 0:
        pass
      else:
        unique_inds = (np.argwhere(batch_det_new[batch, :, :, -1] == unique_vals[k]))
        batch_det_new[batch, unique_inds[:,0],unique_inds[:,1], -1] = consecutive_vals[k]
  return batch_det_new



# training_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training-sort/output_sort11AGE=3IOU=0.15/'
# testing_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/testing-sort/output_sort12AGE=3IOU=0.15/'
optimal_crop_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det/'
save_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_SORT_det/'

training_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training-sort/output_sort9AGE=2IOU=0.3/'
testing_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/testing-sort/output_sort10AGE=2IOU=0.3/'

# ../../Anticipating-Accidents/dataset/features/testing-sort/output_sort10AGE=2IOU=0.3/
# ../../Anticipating-Accidents/dataset/features/training-sort/output_sort9AGE=2IOU=0.3/

fixed_track_ids = (training_path, testing_path)

Create det file to point to optimal crops (run 2_processing_final_outputs.ipynb after).

In [0]:
## take random det file, test bounding box transformation on image
## load det, corresponding files
%matplotlib inline
import numpy as np
import time
import sys
from matplotlib import pyplot as plt
from matplotlib import patches

str_modes = ['training','testing']
train_num = 126
test_num = 46
n_batch = [train_num, test_num ]

ab = 0
feat_type = 'optimal_crops_det_2'
crop_type = 'optimal_crops_det'  #!!!
root_paths = (training_path, testing_path)

for ab in range(2):
    ## Cloud root path:
    # root_path0 = '/home/safon007/share/datasets/Accident_Dataset'
    root_path0 = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
    # for ab in range(len(str_modes)):
    str_mode = str_modes[ab]
    
    frame_path = root_path0+'/dataset/videos/frames/'
    feat_save_path = root_path0+'/preprocessed_features/'+feat_type+'/'+str_mode+'/'
    crop_pointer_path = root_path0+'/preprocessed_features/'+crop_type+'/'+str_mode+'/'
    # track_id_path = fixed_track_ids[ab]
    # root_path = root_path0+'/dataset/features/' +str_mode + '/'
    root_path = root_paths[ab]

    ## Access given batches
    ## Augment/Amend the feature maps with bounding box, distance, optical flow and odometry
    root_num = len(os.listdir(root_path)) 
    n_batchs = np.arange(1,root_num+1)

    tStart_epoch = time.time()
    root_batch_list = os.listdir(root_path) 
    count_batch = 1
    feat_in_root = sort_dir(root_batch_list)
    try:
        feat_in_save = sort_dir(os.listdir(feat_save_path))

    except:
        "Empty features folder!"
        feat_in_save = []

    missing_batches = np.setdiff1d(feat_in_root,feat_in_save)

    if True:
    # if len(missing_batches) != 0:
        for batch in feat_in_root:
            # for batch in missing_batches:
              time_1 = time.time()
              print('Batch: ' + str(count_batch) + ' :: ' + batch)
              #curr_feature = root_batch_list[batch]
              curr_feature = batch
              all_data = np.load(root_path+curr_feature, allow_pickle = True) # initializing the current batch
              # [1]: First try: Adding bounding box locations to the feature map
              # data = all_data['data']
              labels = all_data['labels']
              # det = all_data['det']
              # det = all_data['new_det']
              det = all_data['sortdet']
              ID = all_data['ID']
              frame_paths, frame_labels = retrieve_dir1(labels, ID,str_mode, n_batch[ab])
              pointer_det = shift_indices(np.load(crop_pointer_path+curr_feature)['new_det'])
              new_det = conv_bb_based_off_existing_crop(det, pointer_det, frame_paths)
              args_output = feat_save_path+batch   
              # /content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_SORT_det/training/batch_001.npz

              #!! UNCOMMENT LINE BELOW::
              np.savez(args_output , new_det=new_det)
              print(args_output)
              print(time.time() - time_1)      
              # except:
              #     print("No depth map found for: " + feat_save_path+batch)


Batch: 1 :: batch_001.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/training/batch_001.npz
0.7603573799133301
Batch: 1 :: batch_002.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/training/batch_002.npz
0.7330875396728516
Batch: 1 :: batch_003.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/training/batch_003.npz
0.759876012802124
Batch: 1 :: batch_004.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/training/batch_004.npz
0.7030735015869141
Batch: 1 :: batch_005.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/preprocessed_features/optimal_crops_det_2/training/batch_005.npz
0.6783947944641113
Batch: 1 :: batch_006.npz
/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/prepr

In [0]:
print(glob.glob(frame_paths[1]+'/*.jpg'))
x = np.clip(np.asarray(Image.open( img ), dtype=float) / 255, 0, 1)


# print(os.listdir(frame_paths[1]))

['/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame93.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame4.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame94.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame40.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame11.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame2.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/testing/positive/000490/frame86.jpg', '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/video

In [0]:
# Create dictionary that contains all corrected bounding boxes
# for each batc

## Correctly crop images
#(1): Function retrieves the preferential crop

#(2): Retrieve and save cropped images

#(3): Create masks

## Create depth maps

In [0]:
## take random det file, test bounding box transformation on image
## load det, corresponding files
%matplotlib inline
import numpy as np
import time
import sys
import os
from matplotlib import pyplot as plt
from matplotlib import patches

str_modes = ['training','testing']
train_num = 126
test_num = 46
n_batch = [train_num, test_num ]

ab = 0
feat_type = 'reformatted_det'

for ab in range(2):
    ## Cloud root path:
    # root_path0 = '/home/safon007/share/datasets/Accident_Dataset'
    root_path0 = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents'
    # for ab in range(len(str_modes)):
    str_mode = str_modes[ab]
    root_path = root_path0+'/dataset/features/' +str_mode + '/'
    frame_path = root_path0+'/dataset/videos/frames/'
    feat_save_path = root_path0+'/preprocessed_features/'+feat_type+'/'+str_mode+'/'

    ## Access given batches
    ## Augment/Amend the feature maps with bounding box, distance, optical flow and odometry
    # root_num = len(os.listdir(root_path)) 
    # n_batchs = np.arange(1,root_num+1)

    tStart_epoch = time.time()
    root_batch_list = os.listdir(root_path) 
    count_batch = 1
    feat_in_root = sort_dir(root_batch_list)
    try:
        feat_in_save = sort_dir(os.listdir(feat_save_path))

    except:
        "Empty features folder!"
        feat_in_save = []

    missing_batches = np.setdiff1d(feat_in_root,feat_in_save)

#     if True:
    if len(missing_batches) != 0:
#         for batch in feat_in_root:
            for batch in missing_batches:
              print('Batch: ' + str(count_batch) + ' :: ' + batch)
              #curr_feature = root_batch_list[batch]
              curr_feature = batch
              all_data = np.load(root_path+curr_feature, allow_pickle = True) # initializing the current batch
              # [1]: First try: Adding bounding box locations to the feature map
              # data = all_data['data']
              labels = all_data['labels']
              det = all_data['det']
              ID = all_data['ID']

              try:
                  dist_list = retrieve_distance_dir(labels, ID, str_mode, dist_dir, n_batch[ab])
                  shape_list = list()
                  for k in range(10):
                      shape_list.append(np.load(dist_list[k])['input_shape'])

                  new_det = bounding_box_conversion(det, shape_list)

                  args_output = feat_save_path+batch

                  np.savez(args_output , new_det=new_det)
                  print(args_output)
                    
              except:
                  print("No depth map found for: " + feat_save_path+batch)


NameError: ignored

Get distance features

In [0]:
## note that if we are able to calculate intrinsic camera parameters (pending release), 
#  the google codes have a much better method for getting to global coordinates
def get_XZ(dis_imgs, det):
    output_array = np.zeros((len(dis_imgs),100,19))
    output_features = np.zeros((len(dis_imgs),100,19,2))
    
#     try:
    for ak1 in range(len(dis_imgs)): # retrieve distance matrices
        dis_vid = np.load(dis_imgs[ak1], allow_pickle=True)

        for ak2 in range(100): # retrieve image/frame from video

            crop_type = det[ak1,ak2,:,-2]
            if np.any(crop_type == 1):
                crop_str = 'distance_upper'
            elif np.any(crop_type == 3):
                crop_str = 'distance_lower'
            elif np.any(crop_type == 2):
                crop_str = 'distance_middle'
            else:
                crop_str = ''

            if crop_str == '':
                pass
#                     print('Distance measurement missing!')
            else:
                if np.any(dis_vid[crop_str]) == None:
                    dis_vid1 = dis_vid['distance_middle']
                else:
                    dis_vid1 = dis_vid[crop_str] 
                bound_boxes = det[ak1,ak2,:,:].astype(int) # retrieve current frame from the current video
                num_boxes = bound_boxes.shape[0]
                dis_frame = dis_vid1[ak2,:,:,0]

                ak3_iter = 0
                for ak3 in range(19):
                    if np.sum(bound_boxes[ak3,:]) != 0:
                        c_v = np.floor(dis_frame.shape[1]/2)
    #                     print(bound_boxes)
                        u, v = np.floor((bound_boxes[ak3,1]+bound_boxes[ak3,3])/2), np.floor((bound_boxes[ak3,0]+bound_boxes[ak3,2])/2)
                        if u > 127:
                            u = 127
#                                     print("u out of bounds!!")
                        if v > 415:
                            v = 415
#                                     print("v out of bounds!!")
                        output_features[ak1,ak2,ak3, 1] = dis_frame[int(u), int(v)]
                        f_v = 60 ####!!! Setting horizontal focal length to 2.45 for all videos
                        output_features[ak1,ak2,ak3, 0] = (v-c_v)*(dis_frame[int(u),int(v)]/(f_v))
                        if output_features[ak1,ak2,ak3, 1] == 0:
                            print(dis_frame[int(u), int(v)])
                            print(dis_frame.shape)
                            print('Empty BB')
                            print(ak1,ak2,ak3, u, v)
                    ak3_iter += 1
#                 else:
#                     print('No distance metric')
#     except:
#         print('File not found!')

    return output_features

In [0]:
## take random det file, test bounding box transformation on image
## load det, corresponding files
%matplotlib inline
import numpy as np
import time
import sys
from matplotlib import pyplot as plt
from matplotlib import patches
str_modes = ['training','testing']

##
ab = 0
feat_type = '5_backprop_xz'
###


for ab in range(2):
    ## Cloud root path:
    root_path0 = '/home/safon007/share/datasets/Accident_Dataset'

    # for ab in range(len(str_modes)):
    str_mode = str_modes[ab]
    root_path = root_path0+'/dataset/features/' +str_mode + '/'
    frame_path = root_path0+'/dataset/videos/frames/'
    feat_save_path = root_path0+'/preprocessed_features/'+feat_type+'/'+str_mode+'/'
    det_path = root_path0+'/preprocessed_features/reformatted_det'+'/'+str_mode+'/'

    ## Access given batches
    ## Augment/Amend the feature maps with bounding box, distance, optical flow and odometry
    root_num = len(os.listdir(root_path)) 
    n_batchs = np.arange(1,root_num+1)

    tStart_epoch = time.time()
    root_batch_list = os.listdir(root_path) 
    count_batch = 1
    feat_in_root = sort_dir(root_batch_list)
    try:
        feat_in_save = sort_dir(os.listdir(feat_save_path))

    except:
        "Empty features folder!"
        feat_in_save = []

    missing_batches = np.setdiff1d(feat_in_root,feat_in_save)

#     if True:
    if len(missing_batches) != 0:
#         for batch in feat_in_root:
        for batch in missing_batches:
          print('Batch: ' + str(count_batch) + ' :: ' + batch)
          #curr_feature = root_batch_list[batch]
          curr_feature = batch
          all_data = np.load(root_path+curr_feature, allow_pickle = True) # initializing the current batch
          new_det_data = np.load(det_path+curr_feature, allow_pickle = True)
          # [1]: First try: Adding bounding box locations to the feature map
          data = all_data['data']
          labels = all_data['labels']
          det = new_det_data['new_det']
          ID = all_data['ID']
    
                
          dist_list = retrieve_distance_dir(labels, ID, str_mode, dist_dir, n_batch[ab])
          output_xz = get_XZ(dist_list, det)
 
#           new_det = bounding_box_conversion(det, shape_list)
        
          args_output = feat_save_path+batch
            
          np.savez(args_output , distance=output_xz)
          print('Batch saved!')
                

In [0]:
print(outpdist_listut_xz.shape)
print(outpdist_listut_xz[5,:,1,0])
print(output_xz[7,:,1,1])
print(det[7,:,1,0:4])
print(dist_list[7])
cur_crop = np.load(dist_list[7])['distance_middle']

(10, 100, 19, 2)
[-1.10289243  1.18782821  1.18655083  1.18917255  1.17234548  2.69802544
  2.58577579  2.53766643 -2.65496754 -2.62015508 -2.63814639 -2.69060757
 -2.30516479  0.98382385  0.97761762  0.97979963  0.92201033  0.90937757
  2.50957425  2.5482783   0.83064809 -1.53386658 -1.54682609  0.74847715
  0.70258828  0.64732619  2.4414525   0.60466869  2.42529595 -2.310935
  2.37238637 -0.60258449  0.48996127 -2.36669993 -3.24740582 -0.99086419
 -2.15618286 -1.32430949 -2.9570538   2.0117434  -2.9869159  -2.9979518
 -0.13408759 -1.90745582  1.89736678  1.48707212  0.12456959 -3.04137963
  1.76088334  1.80448748 -2.85363142 -0.01997939  2.02625121 -0.0581193
 -3.11456808 -3.05697895  1.62668187 -2.61140178 -0.30737255 -0.32162879
  1.80483067 -0.36509562 -2.11646354 -2.94144422 -3.07615232 -2.12058582
 -1.69599279 -3.13140421 -3.23503479 -1.51588368 -1.43214172 -2.98315371
 -3.20527836 -3.04098071  0.61646186 -3.11487451  1.03776049 -1.87351656
 -2.9626934  -3.21513216 -3.04574001 -

In [0]:
plt.imshow(cur_crop[11,:,:,0])
print(cur_crop[11,24,51,0])

In [0]:
print(det.shape)
7,:,1

In [0]:
# print(det_reshape.shape)
batch = 8
frame =20
agent = 1

cur_frame = cv2.imread(image_list[batch]+sort_dir(os.listdir(image_list[batch]))[frame] )

dist_map = np.load(dist_list[batch])

file_list = dist_map.files
print(file_list)
shape_list = list()
for k in range(10):
    shape_list.append(np.load(dist_list[k])['input_shape'])

Plotting Code

In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

img = cur_frame

bboxes = det[batch]
new_bboxes = bboxes[frame]
new_bboxes = new_bboxes[~np.all(new_bboxes == 0, axis=1)] ## take out zero values from new_bboxes


img_path = image_list[batch]+sort_dir(os.listdir(image_list[batch]))[frame] 
im = np.array(Image.open(img_path), dtype=np.uint8)

# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(im)
for num_box in range(new_bboxes.shape[0]):
    det1 = det[batch, frame, num_box,:]
    x1, y1 ,x2,y2  = det1[0],det1[1],det1[2],det1[3]
#     x1, y1 ,x2,y2 = bb1[0], bb1[1], bb1[2], bb1[3]
    rect = patches.Rectangle((x1,y1),x2-x1,y2-y1,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    print(det1)

plt.show()

In [0]:
output_base_root = '/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/depth_maps/720_1280_maps/'

# Extracting xz features

Iterate Across batches, creating modified det files *and global xz coordinates

Previous code on extracting global coordinates

In [0]:
def load_images(image_files):
    from skimage.transform import resize
    loaded_images = []
    iter1 = 0

    for file in image_files:
        crop_middle, crop_upper, crop_lower, img_shape = selective_cropping(file)

        if iter1 == 0:
          img_stack_middle, img_stack_upper, img_stack_lower = crop_middle, crop_upper, crop_lower
        elif img_stack_upper.any() != None:
          img_stack_middle, img_stack_upper, img_stack_lower = np.concatenate((img_stack_middle, crop_middle), axis = 0), np.concatenate((img_stack_upper, crop_upper), axis = 0), np.concatenate((img_stack_lower, crop_lower), axis = 0),
        else:
          img_stack_middle = np.concatenate((img_stack_middle, crop_middle), axis = 0)
        iter1+=1


    return img_stack_middle, img_stack_upper, img_stack_lower, img_shape


def crop(img):
    # Perform center cropping, preserving 50% vertically.
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[int(img.shape[0]*(half)):int(img.shape[0]*(1-half)), :]

    # Resize to match target height while preserving aspect ratio.
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))

    # Perform center cropping horizontally.
    remain = b.shape[1] - 416
    # cx /= (b.shape[1]/416)
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]

    return c

def crop_upper(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[0:int(img.shape[0]*(middle_perc)), :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

def crop_lower(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[(-1)*int(img.shape[0]*(middle_perc))::, :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

# random functions
def obtain_keys(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[1]
        key_ls.append(int(integer))
    return(key_ls)
  
def obtain_keys1(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[3]
        key_ls.append(int(integer))
    return(key_ls)
  
def h(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)
  
def sort_dir(lis):
    key_ls = h(obtain_keys(lis))
    return [lis[i] for i in key_ls]

## Perform cropping and resizing, place into network
def selective_cropping(img):
  x = np.clip(np.asarray(Image.open( img ), dtype=float) / 255, 0, 1)
  cropped = np.expand_dims(crop(x), axis = 0)
  cropped1 = None
  cropped2 = None
  if (np.abs(cropped.shape[1]-128)>30) and (np.abs(cropped.shape[2]-416)>30):
    x_1 = np.zeros((720,1280, 3))
    x_1[0:x.shape[0], 0:x.shape[1], :] = x  
    cropped = np.expand_dims(crop(x_1), axis = 0)
    print("Cropping pattern did not fit, resized.")
    print(img_folders)
  else:
    cropped1 = np.expand_dims(crop_upper(x), axis = 0)[:,:,0:-1,:]
    cropped2 = np.expand_dims(crop_lower(x), axis = 0)[:,:,0:-1,:]
  cropped = cropped[:,:,0:-1,:]
  img_shape = x.shape
  return cropped, cropped1, cropped2, img_shape

In [0]:
tf.reset_default_graph() 
train_model = model.Model(data_dir='depth_from_video_in_the_wild/data_example',is_training=True)
saver  = train_model.saver

saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path = 'depth_from_video_in_the_wild/depth_cp_2/model-1000977')

depth_model = model.Model(is_training=False, batch_size = 100)